

##### Global tuberculosis report cover: WHO has published a global tuberculosis (TB) report every year since 1997.The report provides a comprehensive and up-to-date assessment of the TB epidemic, and of progress in prevention, diagnosis and treatment of the disease at global, regional and country levels.


Our goal is to predict the treatment outcome .

References: https://www.who.int/teams/global-tuberculosis-programme/data AND https://www.kaggle.com/datasets/mpwolke/cusersmarildownloadstreatmentcsv




The analysis identified key features that significantly contribute to the variance in the target variable, providing valuable insights for further investigations or decision-making.


Main:
How well can the model predict the treatment success rate for new TB cases using dimensionality reduction and top contributing features?


Our model predicts the treatment success rate for new TB cases with high accuracy and robustness.

 By applying PCA to reduce dimensionality and focusing on the top contributing features, the model explains, on average, 86.75% of the variance in the target variable across different cross-validation folds, with a consistent performance indicated by a low standard deviation of 0.0225. This demonstrates that the model is both accurate and reliable for predicting the treatment success rate, making it a valuable tool for analyzing and predicting outcomes in TB treatment programs.

In [ ]:
try:
  import google.colab
  !{sys.executable} -m pip -q -q install pycm
except:
  pass

import pymc as pm
import arviz as az
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA


Context
Data provided by countries to WHO and estimates of TB burden generated by WHO for the Global Tuberculosis Report

In [ ]:
df = pd.read_csv('treatment.csv', sep=';')

The optimal approach to initiate a data analysis is by addressing missing values first. This step enables us to discern the statistical significance of parameters, which is crucial for our analysis.


In [ ]:
missing_values = df.isnull().sum()

total_values = df.shape[0]

percentage_missing = (missing_values / total_values) * 100

missing_data = pd.DataFrame({'Column': df.columns, 'Percentage Missing': percentage_missing})

missing_data_sorted = missing_data.sort_values(by='Percentage Missing',ascending=False)

print(missing_data_sorted)

In [ ]:
# Drop columns with over 90% missing values
columns_to_drop = missing_data[missing_data['Percentage Missing'] > 90]['Column']
df_cleaned = df.drop(columns_to_drop, axis=1)

# Print the remaining columnsdf_cleaned = df_cleaned.dropna(subset=['year'])
print("Remaining columns after dropping columns with over 90% missing values:")
print(df_cleaned.columns)


In [ ]:
df_cleaned.head()

We can notice that the std for different variables are quite different. A possible approach is to standardize the numerical variables.

In [ ]:
df_cleaned.describe()

In [ ]:
df_cleaned.head()

In [ ]:
columns_to_drop = ['country',	'iso2',	'iso3',	'iso_numeric',	'g_whoregion','year']
columns_to_drop = [col for col in columns_to_drop if col in df_cleaned.columns]

if columns_to_drop:
    df_cleaned = df_cleaned.drop(columns=columns_to_drop)

object_columns = df_cleaned.select_dtypes(include=['object']).columns

print("Remaining object columns:")
print(object_columns)



In [ ]:
# Compute the correlation matrix on the standardized numeric columns
numeric_df = df_cleaned.select_dtypes(include=[np.number])
corr_matrix = numeric_df.corr()

# Plot the heatmap
#plt.figure(figsize=(20, 10))
#sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
#plt.show()
corr_matrix

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np
# Scale the selected features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_cleaned)

# Check for NaNs
if np.isnan(X_scaled).any():
    imputer = SimpleImputer(strategy='mean')  # You can choose another strategy
    X_scaled = imputer.fit_transform(X_scaled)

# Confirm no more NaNs
assert not np.isnan(X_scaled).any(), "NaNs remain in the data after imputing"


In [ ]:
# Perform PCA
pca = PCA(n_components=None)  # None means all components are kept
X_pca = pca.fit_transform(X_scaled)

# Create a DataFrame with principal components
pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
df_pca = pd.DataFrame(X_pca, columns=pca_columns)

# Print explained variance ratio
explained_variance = pca.explained_variance_ratio_
print("Explained variance ratio by each principal component:")
print(explained_variance)

# Visualize the cumulative explained variance
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(explained_variance) + 1), explained_variance.cumsum(), marker='o')
plt.title('Cumulative Explained Variance by Principal Components')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance')
plt.grid()
plt.show()

**Note**: This approach may lack significance as there is no assurance that all the data shares the same units. Hence, to ensure a more meaningful analysis, employing standardscaler and PCA (Principal Component Analysis) is essential and stan.

Principal Component Analysis (PCA) is a dimensionality reduction technique used to transform high-dimensional data into a lower-dimensional space while preserving most of the original information. It achieves this by identifying the directions, called principal components, along which the data varies the most.


Note: While we have already established the reduced dimensions through an analysis of the percentage of NaN values, in this particular step, our focus lies solely on applying the method rather than revisiting those previous results.







In [ ]:
var_cumu = np.cumsum(pca.explained_variance_ratio_)
fig = plt.figure(figsize=[12,8],dpi=200)
plt.vlines(x=4, ymax=1, ymin=0, colors="r", linestyles="--")
plt.hlines(y=0.95, xmax=30, xmin=0, colors="g", linestyles="--")
plt.plot(var_cumu)
plt.ylabel("Cumulative variance explained")
plt.show()

In [ ]:
df_pca.head()

From the graphs, we identify the point where the cumulative explained variance reaches approximately 95%. This is the elbow point where adding more components doesn't significantly increase the explained variance.

In this case, the elbow point is around 20 components.

In [ ]:
# Select the number of components that capture around 95% of the variance
n_components = np.argmax(explained_variance.cumsum() >= 0.95) + 1
print(f"Number of components to retain: {n_components}")

pca = PCA(n_components=n_components)
X_pca_selected = pca.fit_transform(X_scaled)

# Create a DataFrame with principal components
pca_columns = [f'PC{i+1}' for i in range(X_pca_selected.shape[1])]
df_pca_selected = pd.DataFrame(X_pca_selected, columns=pca_columns)

In [ ]:
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
loading_matrix = pd.DataFrame(loadings, columns=pca_columns)

print("PCA Loadings (how each original feature contributes to the principal components):")
print(loading_matrix)

plt.figure(figsize=(12, 8))
loading_matrix.abs().plot(kind='bar', figsize=(15, 7))
plt.title('PCA Loadings (Absolute Values) - Feature Contributions to Principal Components')
plt.xlabel('Original Features')
plt.ylabel('Loading Value')
plt.legend(title='Principal Components')
plt.grid(True)
plt.show()


In [ ]:
top_n = 5
top_features = pd.DataFrame()

for i in range(n_components):
    pc = f'PC{i+1}'
    top_features[pc] = loading_matrix[pc].abs().nlargest(top_n).index

print("Top contributing features for each principal component:")
print(top_features)

# Visualize the top contributing features
top_features_plot = loading_matrix.loc[top_features.values.flatten()].abs().plot(kind='bar', figsize=(15, 7))
plt.title('Top Contributing Features to Principal Components')
plt.xlabel('Original Features')
plt.ylabel('Loading Value')
plt.legend(title='Principal Components')
plt.grid(True)
plt.show()


Model Refinement Using Top Contributing Features


In [ ]:
imputer = SimpleImputer(strategy='mean')
df_cleaned_imputed = pd.DataFrame(imputer.fit_transform(df_cleaned), columns=df_cleaned.columns)

target_variable = 'c_new_tsr'

if target_variable not in df_cleaned.columns:
    raise ValueError(f"Target variable '{target_variable}' not found in the DataFrame.")

features = df_cleaned_imputed.drop(columns=[target_variable])
target = df_cleaned_imputed[target_variable]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)


In [ ]:
# Perform PCA
pca = PCA()
pca.fit(X_scaled)
explained_variance = pca.explained_variance_ratio_

# Select the number of components that capture around 95% of the variance
n_components = np.argmax(explained_variance.cumsum() >= 0.95) + 1
print(f"Number of components to retain: {n_components}")

pca = PCA(n_components=n_components)
X_pca_selected = pca.fit_transform(X_scaled)

pca_columns = [f'PC{i+1}' for i in range(X_pca_selected.shape[1])]
df_pca_selected = pd.DataFrame(X_pca_selected, columns=pca_columns)

#PCA-transformed DataFrame for verification
print("DataFrame with PCA-transformed components:")
print(df_pca_selected.head())

# Inspect PCA Loadings
loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
loading_matrix = pd.DataFrame(loadings, columns=pca_columns, index=features.columns)

print("PCA Loadings (how each original feature contributes to the principal components):")
print(loading_matrix)



We have tested our model for N=19 features and it does not increase the accuracy of the model. For this reason, we choose 5 features

In [ ]:
# Top contributing features
top_n = 5
top_features = pd.DataFrame()

for i in range(n_components):
    pc = f'PC{i+1}'
    top_features[pc] = loading_matrix[pc].abs().nlargest(top_n).index

print("Top contributing features for each principal component:")
print(top_features)

# Flatten the list of top features and remove duplicates
top_features_flat = top_features.values.flatten()
top_features_set = list(set(top_features_flat))

# Create a DataFrame with the features
top_features_df = features[top_features_set]


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(top_features_df, target, test_size=0.2, random_state=42)

#Train a model (RandomForestRegressor )
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Model Mean Squared Error with top contributing features: {mse}")
print(f"Model R^2 Score with top contributing features: {r2}")

In [ ]:
from sklearn.model_selection import cross_val_score

# Perform cross-validation
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='r2')

print(f"Cross-validated R^2 scores: {cv_scores}")
print(f"Mean cross-validated R^2 score: {np.mean(cv_scores)}")
print(f"Standard deviation of cross-validated R^2 score: {np.std(cv_scores)}")


In [ ]:
feature_importances = model.feature_importances_
feature_names = top_features_set

# Create a DataFrame for visualization
feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Plot feature importances
plt.figure(figsize=(12, 8))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'])
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Feature Importances in RandomForestRegressor')
plt.gca().invert_yaxis()
plt.show()


We applied Principal Component Analysis (PCA) to reduce the dimensionality of the dataset. Retained 19 principal components that capture around 95% of the variance in the data.


What happens if our model if we select features with compatible std?

Bayesian inference model using machine learning for predicting the treatment success rate from our dataset, I'll follow several steps. These steps involve preparing our data, selecting a model, and implementing the Bayesian approach. This model allows you to incorporate prior beliefs about the parameters and update these beliefs based on the data.

In [ ]:
df_cleaned_nonan = df_cleaned.dropna(subset=['c_new_tsr'])

X = df_cleaned_nonan[['rep_meth', 'new_sp_coh', 'new_sp_cur', 'new_sp_cmplt', 'new_sp_died', 'new_sp_fail', 'new_sp_def', 'c_new_sp_tsr', 'c_ret_tsr']].values
y = df_cleaned_nonan['c_new_tsr'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

print("NaN values in X_train_imputed:", np.isnan(X_train_imputed).sum())
print("Inf values in X_train_imputed:", np.isinf(X_train_imputed).sum())

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

# Check for NaN or Inf values after standardization
print("NaN values in X_train_scaled:", np.isnan(X_train_scaled).sum())
print("Inf values in X_train", np.isnan(X_train_scaled).sum())


In [ ]:
with pm.Model() as model:
    x_shared = pm.MutableData('x', X_train_scaled)
    y_shared = pm.MutableData('y', y_train)

    # Priors for unknown model parameters
    alpha = pm.Normal('alpha', mu=0, sigma=10)
    beta = pm.Normal('beta', mu=0, sigma=10, shape=X_train_scaled.shape[1])
    sigma = pm.HalfNormal('sigma', sigma=1)

    # Expected value of outcome
    mu = alpha + pm.math.dot(x_shared, beta)

    # Likelihood (sampling distribution) of observations
    obs = pm.Normal('obs', mu=mu, sigma=sigma, observed=y_shared)

    # Sample from the posterior using Markov chain Monte Carlo
    trace = pm.sample(1000, tune=2000, return_inferencedata=True, init='adapt_diag')

In [ ]:
az.plot_trace(trace)

# Summary statistics for the posterior distributions
summary = az.summary(trace)
print(summary)


In [ ]:
with model:
    pm.set_data({'x': X_test_scaled, 'y': y_test})
    post_pred = pm.sample_posterior_predictive(trace, var_names=["obs"], random_seed=42)

y_obs_samples = post_pred.posterior_predictive['obs']

mean_prediction = y_obs_samples.mean(axis=(0, 1))
lower_bound = np.percentile(y_obs_samples, 2.5, axis=(0, 1))
upper_bound = np.percentile(y_obs_samples, 97.5, axis=(0, 1))

mse_test = mean_squared_error(y_test, mean_prediction)
r2_test = r2_score(y_test, mean_prediction)
print(f"Test set Mean Squared Error: {mse_test}")
print(f"Test set R^2 Score: {r2_test}")

# Plotting the results
plt.figure(figsize=(10, 6))
plt.scatter(range(len(y_test)), y_test, label='Actual')
plt.scatter(range(len(mean_prediction)), mean_prediction, color='red', label='Predicted Mean')
plt.fill_between(range(len(y_test)), lower_bound, upper_bound, color='purple', alpha=0.2, label='95% Credible Interval')
plt.legend()
plt.xlabel('Observation Index')
plt.ylabel('Treatment Success Rate')
plt.title('Actual vs. Predicted Treatment Success Rates on Test Set')
plt.show()